In [1]:
import pandas as pd
import glob
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time
import urllib.parse
import base64
from urllib.parse import urlparse, parse_qs
import json

# Data Allocation

In [2]:
folder_path = "C:\\Users\\Pratt\\Desktop\\HKUST-RA\\[-Ongoing-] Text analysis in Glassdoor data\\Basic Data\\glassdoor-ep-revirew"
csv_files = []
df_list = []

In [3]:
# 获取所有CSV文件
for item in os.listdir(folder_path):
    if item.endswith('.csv'):
        full_path = os.path.join(folder_path, item)
        csv_files.append(full_path)

In [4]:
success_count = 0
failed_count = 0

# 使用普通循环，不使用low_memory=False
for file_path in tqdm(csv_files, desc="📊 读取CSV文件", unit="文件"):
    try:
        # 不使用low_memory=False，使用默认设置
        df = pd.read_csv(file_path)
        df_list.append(df)
        success_count += 1
    except Exception as e:
        failed_count += 1
        # 不中断tqdm，只记录错误
        tqdm.write(f"❌ 读取失败: {os.path.basename(file_path)} - {str(e)[:50]}...")
        continue

📊 读取CSV文件: 100%|██████████| 76/76 [01:57<00:00,  1.55s/文件]


In [5]:
df_full = pd.concat(df_list, ignore_index=True)

In [6]:
group_column = "公司ID"  # 替换为你的列名
new_df = (df_full[df_full.groupby(group_column)[group_column].transform('size') >= 100]
          .groupby(group_column)
          .first()
          .reset_index())

In [7]:
df_list_1 = new_df['公司名'].tolist()
df_list_2 = new_df['公司网址'].tolist()
df_list_3 = new_df['公司简介'].tolist()
df = pd.DataFrame({'company': df_list_1, 'url': df_list_2, 'intro': df_list_3})
results = []

In [8]:
for i in range(10):
    max_retries = 5
    retry_count = 0
    found_url = None
    company = df.loc[i, 'company']
    url = df.loc[i, 'url']
    intro = df.loc[i, 'intro']
    
    while retry_count < max_retries and not found_url:
        options = uc.ChromeOptions()

        options.add_argument('--disable-blink-features=AutomationControlled')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-gpu')
        options.add_argument('--disable-infobars')
        options.add_argument('--window-size=1920,1080')
        options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')

        driver = uc.Chrome(options=options)
        time.sleep(2)

        # ------------------------------------------------------------------------------ # Bing搜索
        try:           
            driver.get("https://www.bing.com/")
            time.sleep(2.5)
            
            # 找到搜索框元素并输入查询
            search_box = driver.find_element(By.NAME, "q")
            search_query = f'{company} glassdoor {intro[:10]}'
            search_box.clear()
            search_box.send_keys(search_query)
            time.sleep(1.5)
            
            # 提交搜索（按回车键）
            search_box.send_keys(Keys.RETURN)
            time.sleep(4)

            result_links = driver.find_elements(By.CSS_SELECTOR, "h2 a")
            time.sleep(1)
            current_found_url = None

            for i, link in enumerate(result_links, 1):
                decoded_url = link.get_attribute("href")
                parsed_url = urlparse(decoded_url)
                # 检查是否包含bing.com
                if "www.bing.com" in decoded_url:
                    u_param = parse_qs(parsed_url.query).get('u', [None])[0]
                    if not u_param:
                        continue
                    try:
                        b64_str = u_param[2:]
                        padding = len(b64_str) % 4
                        if padding:
                            b64_str += "=" * (4 - padding)
                        decoded_bytes = base64.urlsafe_b64decode(b64_str)
                        decoded_url = decoded_bytes.decode('utf-8')
                    except Exception as e:
                        continue
                try:
                    # 第一步：筛选glassdoor链接
                    if "glassdoor" not in decoded_url:
                        continue
                    # 第二步：先筛选Overview链接
                    if "/Overview/" in decoded_url:
                        current_found_url = decoded_url
                        break
                    # 第三步：如果没有Overview，筛选Reviews链接
                    elif "/Reviews/" in decoded_url:
                        current_found_url = decoded_url
                        break
                except Exception as e:
                    continue
            
            if current_found_url:
                found_url = current_found_url
                break
                
        except Exception as e:  
            print(f"Error processing {company} on retry {retry_count + 1}: {e}")
        
        finally:
            # 确保每次尝试后都关闭driver
            driver.quit()
            time.sleep(1)
        
        retry_count += 1
        
        # 如果不是最后一次重试，等待一会儿再重试
        if retry_count < max_retries and not found_url:
            time.sleep(2)  # 重试前等待时间
    
    # 根据最终结果决定添加什么
    if found_url:
        results.append({'company': company, 'url': found_url})
        print(f"Successfully found URL for {company}")
    else:
        results.append({'company': company, 'url': None})
        print(f"Failed to find valid Glassdoor link for {company} after {max_retries} attempts")

Successfully found URL for AAR
Successfully found URL for AMETEK
Successfully found URL for American Airlines
Successfully found URL for Abbott
Successfully found URL for BankUnited
Successfully found URL for AMD
Successfully found URL for Aetna
Successfully found URL for Air Products
Successfully found URL for Airgas
Successfully found URL for Alaska Airlines


In [9]:
# 保存为 JSON 文件
with open('C:\\Users\\Pratt\\Desktop\\HKUST-RA\\[-Ongoing-] Text analysis in Glassdoor data\\Url-Scrap-Method-2-Glassdoor-ep\\Urls-2.1.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)